# Lab 9: Design of Experiment
In this lab, we will observe the effect of the DoE in the Bayesian optimization and Bio-inspired approaches.

In the following (hidden) block, the utilities used for running the experiments are implemented.

The list of available benchmark functions can be found at this [link](https://gitlab.com/luca.baronti/python_benchmark_functions)

**NOTE**: When studying the effect of the parameters is *extremely* important to vary just one parameter at a time. Therefore, you are suggested to study one parameter by fixing all the others, and then moving to the next.

Moreover, when comparing different algorithms, is *very* important to run each of them several times (e.g., 30) by using different initial random seeds.


You will use the sampling technique seen in the lecture. In particular, you will have to implement:


*   Random sampling
*   The Halton sequence
*   The full factorial sampling
*   The Latin Hypercube sampling



In [ ]:
!pip install cma
!pip install inspyred
!pip install benchmark_functions
!pip install scikit-optimize

Code for the Bayesian Optimization, you have to reuse the code from Lab. 6 for the acquisition and the objective functions.

In [ ]:
#@title

# example of bayesian optimization for a 1d function from scratch
from math import sin
from math import pi
from numpy import arange
from numpy import vstack
from numpy import argmax
from numpy import asarray
from numpy.random import normal
from numpy.random import random
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from warnings import catch_warnings
from warnings import simplefilter
from matplotlib import pyplot

from numpy import mean
from sklearn.datasets import make_blobs
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from skopt.space import Integer
from skopt.utils import use_named_args
from warnings import catch_warnings
from skopt import gp_minimize
from warnings import simplefilter


# surrogate or approximation for the objective function
def surrogate(model, X):
    # catch any warning generated when making a prediction
    with catch_warnings():
        # ignore generated warnings
        simplefilter("ignore")
        return model.predict(X, return_std=True)


def acquisition(X, Xsamples, model):
    # # # calculate the best surrogate score found so far
    # yhat, _ = surrogate(model, X)
    # print(X)
    # best = max(yhat)
    # # calculate mean and stdev via surrogate function
    # mu, std = surrogate(model, Xsamples)
    # mu = mu[:, 0]
    # # calculate the probability of improvement
    probs = acquisition_function(X, Xsamples, model)
    return probs


# optimize the acquisition function
def opt_acquisition(X, y, model):
    # random search, generate random samples
    Xsamples = random(100)
    Xsamples = Xsamples.reshape(len(Xsamples), 1)
    # calculate the acquisition function for each sample
    scores = acquisition(X, Xsamples, model)
    # locate the index of the largest scores
    ix = argmax(scores)
    return Xsamples[ix, 0]


# plot real observations vs surrogate function
def plot(X, y, model):
    # scatter plot of inputs and real objective function
    pyplot.scatter(X, y)
    # line plot of surrogate function across domain
    Xsamples = asarray(arange(0, 1, 0.001))
    Xsamples = Xsamples.reshape(len(Xsamples), 1)
    ysamples, _ = surrogate(model, Xsamples)
    pyplot.plot(Xsamples, ysamples)
    # show the plot
    pyplot.show()


def bayesianOptmization(generation):
    # sample the domain sparsely with noise
    X, y = initial_point(size=2)
    # reshape into rows and cols
    X = X.reshape(len(X), 1)
    y = y.reshape(len(y), 1)
    # define the model
    model = GaussianProcessRegressor()  #you can set the kernel and the optmizer
    # fit the model
    model.fit(X, y)
    # perform the optimization process
    for i in range(generation):
        # select the next point to sample
        x = opt_acquisition(X, y, model)
        # sample the point
        actual = objective(x)
        # summarize the finding
        est, _ = surrogate(model, [[x]])
        # add the data to the dataset
        X = vstack((X, [[x]]))
        y = vstack((y, [[actual]]))
        # update the model
        model.fit(X, y)
    return X, y, model


# objective function
def objective(x, noise=0.1):
    return x + normal(loc=0, scale=noise)


#remember to return the value in the right order and type.
def initial_point(generator_type, random, args):
    X = generator(generator_type)(random, args)
    Y = asarray([objective(x) for x in X])
    return X, Y


# you have to add the parameters that you need.
def acquisition_function(X, Xsamples, model):
    # calculate the best surrogate score found so far
    yhat, _ = surrogate(model, X)
    best = max(yhat)
    # calculate mean and stdev via surrogate function
    mu, std = surrogate(model, Xsamples)
    mu = mu[:, 0]
    # calculate the probability of improvement
    probs = norm.cdf((mu - best) / (std + 1E-9))
    return probs

Code for the Genetic algorithm from lab. 7

In [ ]:
#@title
import cma
import inspyred
import importlib
import functools
import numpy as np
from pylab import *
from inspyred import ec
from copy import deepcopy
from functools import reduce
import benchmark_functions as bf
from matplotlib import pyplot as plt
from inspyred.ec import EvolutionaryComputation
from inspyred.ec import selectors, replacers, terminators

GLOBAL = 'Global'
INDIVIDUAL = 'Individual'
CORRELATED = 'Correlated'
STAR = 'star'
RING = 'ring'


class OptFun():
    def __init__(self, wf):
        self.f = wf
        self.history = []
        self.__name__ = f'OptFun({wf.__class__})'

    def __call__(self, candidates, *args, **kwargs):
        y = []
        for x0 in candidates:
            self.history.append(deepcopy(x0))
            y.append(self.f(x0))
        return y

    def minima(self):
        return self.f.minima()

    def bounder(self):
        def fcn(candidate, *args):
            bounds = self.f.suggested_bounds()

            for i, (m, M) in enumerate(zip(*bounds)):
                if candidate[i] < m:
                    candidate[i] = m
                if candidate[i] > M:
                    candidate[i] = M
            return candidate

        return fcn

    def bounds(self):
        return self._convert_bounds(self.f.suggested_bounds())

    def heatmap(self, fn=None):
        plt.clf()
        resolution = 50
        fig = plt.figure()
        fig.canvas.set_window_title('Benchmark Function: ' + self.f._name)
        fig.suptitle(self.f._name)
        bounds_lower, bounds_upper = self.f.suggested_bounds()
        x = np.linspace(bounds_lower[0], bounds_upper[0], resolution)
        if self.f._n_dimensions > 1:
            y = np.linspace(bounds_lower[1], bounds_upper[1], resolution)
            X, Y = np.meshgrid(x, y)
            Z = np.asarray([[self.f((X[i][j], Y[i][j])) for j in range(len(X[i]))] for i in range(len(X))])

        plt.contour(x, y, Z, 15, linewidths=0.5, colors='k')  # hight lines
        plt.contourf(x, y, Z, 15, cmap='viridis', vmin=Z.min(), vmax=Z.max())  # heat map
        plt.xlabel('x')
        plt.ylabel('y')
        cbar = plt.colorbar()
        cbar.set_label('z')
        if len(self.history) > 0:  # plot points
            xdata = [x[0] for x in self.history]
            ydata = [x[1] for x in self.history]
            plt.plot(xdata, ydata, "or-", markersize=2, linewidth=2)
        if fn is None:
            plt.show()
        else:
            plt.savefig(fn, dpi=400)

    def plot(self):
        plt.clf()
        values = [self.f(v) for v in self.history]
        min = func.minima()[0].score
        plt.plot(values)
        plt.axhline(min, color="r", label="optimum")
        plt.legend()
        plt.show()

    def _convert_bounds(self, bounds):
        new_bounds = []
        for i in range(len(bounds[0])):
            new_bounds.append((bounds[0][i], bounds[1][i]))
        return new_bounds

    def current_calls(self):
        return len(self.history)


def choice_without_replacement(rng, n, size):
    result = set()
    while len(result) < size:
        result.add(rng.randint(0, n))
    return result


class NumpyRandomWrapper(RandomState):
    def __init__(self, seed=None):
        super(NumpyRandomWrapper, self).__init__(seed)

    def sample(self, population, k):
        if isinstance(population, int):
            population = range(population)

        return asarray([population[i] for i in
                        choice_without_replacement(self, len(population), k)])
        #return #self.choice(population, k, replace=False)

    def random(self):
        return self.random_sample()

    def gauss(self, mu, sigma):
        return self.normal(mu, sigma)


def initial_pop_observer(population, num_generations, num_evaluations,
                         args):
    if num_generations == 0:
        args["initial_pop_storage"]["individuals"] = asarray([guy.candidate
                                                              for guy in population])
        args["initial_pop_storage"]["fitnesses"] = asarray([guy.fitness
                                                            for guy in population])


def generator(case):
    if case == "random":
        return random_generator
    if case == "LHS":
        return lhs_generator
    if case == "Halton":
        return Halton_generator
    if case == "FF":
        return ff_generator


def generator_wrapper(func):
    @functools.wraps(func)
    def _generator(random, args):
        return asarray(func(random, args))

    return _generator


def single_objective_evaluator(candidates, args):
    problem = args["problem"]
    return [CombinedObjectives(fit, args) for fit in
            problem.evaluator(candidates, args)]


def run_ga(random, generator_type, func, num_vars=0,
           maximize=False, **kwargs):
    #create dictionaries to store data about initial population, and lines
    initial_pop_storage = {}

    algorithm = ec.EvolutionaryComputation(random)
    algorithm.terminator = ec.terminators.generation_termination
    algorithm.replacer = ec.replacers.generational_replacement
    algorithm.variator = [ec.variators.uniform_crossover, ec.variators.gaussian_mutation]
    algorithm.selector = ec.selectors.tournament_selection

    algorithm.observer = initial_pop_observer

    kwargs["num_selected"] = kwargs["pop_size"]

    kwargs["bounder"] = func.bounder()
    kwargs["generator"] = generator(generator_type)

    final_pop = algorithm.evolve(evaluator=func,
                                 maximize=False,
                                 initial_pop_storage=initial_pop_storage,
                                 num_vars=num_vars,
                                 **kwargs)

    #best_guy = final_pop[0].candidate
    #best_fitness = final_pop[0].fitness
    final_pop_fitnesses = asarray([guy.fitness for guy in final_pop])
    final_pop_candidates = asarray([guy.candidate for guy in final_pop])

    sort_indexes = sorted(range(len(final_pop_fitnesses)), key=final_pop_fitnesses.__getitem__)
    final_pop_fitnesses = final_pop_fitnesses[sort_indexes]
    final_pop_candidates = final_pop_candidates[sort_indexes]

    best_guy = final_pop_candidates[0]
    best_fitness = final_pop_fitnesses[0]

    return best_guy, best_fitness, final_pop


Samplings methods to implement

In [ ]:
def random_generator(random, args):
    pass


def lhs_generator(random, args):
    pass


def Halton_generator(random, args):
    pass


def ff_generator(random, args):
    pass

# Exercise 1: Bayesian Optmization

In [ ]:
X, y, model = bayesianOptmization(1)
plot(X, y, model)
# best result
ix = argmax(y)
print('Best Result: x=%.3f, y=%.3f' % (X[ix], y[ix]))

# Exercise 2: Genetic Algorithm

In [ ]:
from random import Random

func = OptFun(bf.Ackley(2))

args = {}
args["num_vars"] = 2  # Number of dimensions of the search space
args["gaussian_stdev"] = 1.0  # Standard deviation of the Gaussian mutations
args["tournament_size"] = 2
args["num_elites"] = 1  # number of elite individuals to maintain in each gen
args["pop_size"] = 20  # population size
args["pop_init_range"] = func.bounds()[0]  # Range for the initial population
args["max_generations"] = 50  # Number of generations of the GA
args["crossover_rate"] = 0.0
args["mutation_rate"] = 1.0

run_ga(
    Random(0),  # Seeded random number generator
    func,
    **args
)

func.heatmap()
func.plot()

In this lab, you will need to compare the algorithms studied in the previous lessons with their version enhanced with different DOE techniques.

1.   How do the performances increases? Are the algorithms faster to converge, or can they find better solutions? 
2.   Is there an approach better than the others in terms of performance?
3.   How much do the DOEs affect the search cost? 
